In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# URL base de las noticias de fact-check en Verificat
BASE_URL = "https://www.verificat.cat/es/fact-check-politica/"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Configuración del driver de Selenium
def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(options=options)

driver = initialize_driver()
driver.get(BASE_URL)
wait = WebDriverWait(driver, 10)
actions = ActionChains(driver)

# Función para cargar y analizar el contenido de la página

def get_page_soup():
    page_source = driver.page_source
    return BeautifulSoup(page_source, 'html.parser')

# Función para pasar a la siguiente página

def go_to_next_page(current_page):
    try:
        # Ajuste del selector XPath para encontrar el botón de la página siguiente correctamente
        next_page_button = wait.until(EC.element_to_be_clickable((By.XPATH, f"/html/body/div[1]/div/div/main/article/div[2]/div/div[4]/ul/li/a[text()='{current_page + 1}']")))
        actions.move_to_element(next_page_button).perform()  # Mover el cursor al botón
        time.sleep(2)  # Esperar medio segundo
        next_page_button.click()  # Hacer clic en el botón
        time.sleep(5)  # Esperar 5 segundos para que se cargue la siguiente página
    except (TimeoutException, NoSuchElementException):
        print("No se pudo encontrar o hacer clic en el botón de la página siguiente. Puede que no haya más páginas disponibles.")
        return False
    return True

# Función para extraer detalles de cada noticia

def get_article_details(article_url):
    try:
        response = requests.get(article_url, headers=HEADERS)
        response.raise_for_status()
        page_content = response.content
        
        if not page_content:
            return "No se encontró el título", "No se encontró la fecha", "No se encontró el contenido"

        soup = BeautifulSoup(page_content, 'html.parser')
        # Obtener el título
        title_tag = soup.find("h1", class_="entry-title")
        title = title_tag.get_text(strip=True) if title_tag else "No se encontró el título"

        # Obtener la fecha
        date_tag = soup.find("time")
        date = date_tag["datetime"] if date_tag and date_tag.has_attr("datetime") else "No se encontró la fecha"

        # Obtener el contenido de la noticia
        content_elements = soup.select("div.entry-content > p")
        content = " ".join([elem.get_text(strip=True) for elem in content_elements])
        content = content if content else "No se encontró el contenido"
        time.sleep(5)
        return title, date, content
    except (requests.RequestException, AttributeError) as e:
        print(f"Error al obtener detalles del artículo: {e}")
        return "No se encontró el título", "No se encontró la fecha", "No se encontró el contenido"

# Almacenar noticias
news_data = []
news_counter = 0
saved_titles = set()  # Mantener un registro de los títulos ya guardados
current_page = 1

try:
    # Crear el archivo CSV y agregar los encabezados
    with open('verificat.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Número de Noticia", "Titular", "Fecha", "Contenido"])

    while current_page <= 30:
        # Crear el objeto BeautifulSoup a partir del contenido actual de la página
        soup = get_page_soup()

        # Obtener los enlaces de las noticias
        articles = []
        article_cards = soup.select("div.ultp-block-content > h3 > a")
        for link in article_cards:
            try:
                if link and link['href'] not in saved_titles:
                    articles.append(link['href'])
            except AttributeError:
                continue

        if not articles:
            print("No se encontraron más artículos.")
            break

        for article_url in articles:
            title, date, content = get_article_details(article_url)

            # Verificar si el título ya ha sido guardado
            if title in saved_titles:
                print(f"Noticia duplicada encontrada: {title}, omitiendo...")
                continue

            news_counter += 1
            saved_titles.add(title)
            news_data.append([news_counter, title, date, content])

            # Cada 10 noticias, guardar en CSV
            if news_counter % 10 == 0:
                with open('verificat.csv', mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerows(news_data)
                news_data = []  # Limpiar datos almacenados
                print("Noticias guardadas")

        # Guardar las noticias después de cada carga
        with open('verificat.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerows(news_data)
        news_data = []  # Limpiar datos almacenados
        print(f"Guardadas las noticias de la página actual: {news_counter}")
        time.sleep(5)

        # Intentar ir a la siguiente página
        if not go_to_next_page(current_page):
            break

        current_page += 1

except Exception as e:
    print(f"Error durante la ejecución: {e}")

finally:
    # Guardar las noticias restantes
    if news_data:
        with open('verificat.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerows(news_data)

    # Cerrar el driver
    driver.quit()
    print("Ejecución finalizada y datos guardados correctamente.")

Noticias guardadas
Guardadas las noticias de la página actual: 10
Noticias guardadas
Guardadas las noticias de la página actual: 20
Noticias guardadas
Guardadas las noticias de la página actual: 30
Noticias guardadas
Guardadas las noticias de la página actual: 40
Noticias guardadas
Guardadas las noticias de la página actual: 50
Noticias guardadas
Guardadas las noticias de la página actual: 60
Noticias guardadas
Guardadas las noticias de la página actual: 70
Noticias guardadas
Guardadas las noticias de la página actual: 80
Noticias guardadas
Guardadas las noticias de la página actual: 90
Noticias guardadas
Guardadas las noticias de la página actual: 100
Noticias guardadas
Guardadas las noticias de la página actual: 110
Noticias guardadas
Guardadas las noticias de la página actual: 120
Noticias guardadas
Guardadas las noticias de la página actual: 130
Noticia duplicada encontrada: La Fiscalía no pide la misma pena para Laura Borràs que la prevista por asesinato, sino menos de la mitad de